https://blog.csdn.net/xmuecor/article/details/78383352

In [8]:
import talib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pdb

high = np.random.random(100)*100
low = np.random.random(100)*100

#high = np.arange(100).astype(float)
#low = np.arange(100).astype(float) - 1


In [41]:

def dm_one_period(high, prehigh, low, prelow):
    deltahigh = high - prehigh
    deltalow = prelow - low
    
    if (deltahigh < 0 and deltalow < 0) or deltahigh == deltalow:
        plusdm = 0
        minusdm = 0
    elif deltahigh > deltalow:
        plusdm = deltahigh
        minusdm = 0
    elif deltahigh < deltalow:
        plusdm = 0
        minusdm = deltalow
    else:
        raise ExceptionError
        
    return plusdm, minusdm

def SAR_inc(xp, af, preSAR):
    sar = af*(xp - preSAR) + preSAR
    
    return sar

def SAR(high, low, acceleration=0, maximum=0):
    sar = np.full_like(high, fill_value=np.nan)
    direction = [''] * len(high)
    xp = None
    af = acceleration
    
    for i in range(len(high)):
        if i == 0:
            pass
        elif i == 1:
            dm_plus, dm_minus = dm_one_period(high[i], high[i-1], low[i], low[i-1])
            direction[i] = 'long' if dm_plus >= dm_minus else 'short'
            
            if direction[i] == 'long':
                sar[i] = low[i-1]
                xp = high[i-1]
            elif direction[i] == 'short':
                sar[i] = high[i-1]
                xp = low[i-1]
            else:
                raise ExceptionError
        else:
            # 判断当前方向
            if direction[i-1] == 'long':
                # 反转
                if low[i] <= sar[i-1]:
                    # 跌势第一步
                    direction[i] = 'short'
                    sar[i] = xp
                    if sar[i] < high[i-1]:
                        sar[i] = high[i-1]
                    if sar[i] < high[i]:
                        sar[i] = high[i]
                    
                    af = acceleration
                    xp = low[i]
                    sar[i] = sar[i] + af*(xp - sar[i])
                    if sar[i] < high[i-1]:
                        sar[i] = high[i-1]
                    if sar[i] < high[i]:
                        sar[i] = high[i]
                else:
                    # 涨势第二步
                    direction[i] = 'long'
                    if high[i] > xp:
                        xp = high[i]
                        af += acceleration
                        if af > maximum:
                            af = maximum
                    sar[i] = sar[i-1] + (xp - sar[i-1])*af
                    if sar[i] > low[i-1]:
                        sar[i] = low[i-1]
                    if sar[i] > low[i]:
                        sar[i] = low[i]
            elif direction[i-1] == 'short':
                # 反转
                if high[i] >= sar[i-1]:
                    # 涨势第一步
                    direction[i] = 'long'
                    sar[i] = xp
                    if sar[i] > low[i-1]:
                        sar[i] = low[i-1]
                    if sar[i] > low[i]:
                        sar[i] = low[i]
                        
                    af = acceleration
                    xp = high[i]
                    sar[i] = sar[i] + af*(xp - sar[i])
                    if sar[i] > low[i-1]:
                        sar[i] = low[i-1]
                    if sar[i] > low[i]:
                        sar[i] = low[i]
                else:
                    # 跌势第二步
                    direction[i] = 'short'
                    if low[i] < xp:
                        xp = low[i]
                        af += acceleration
                        if af > maximum:
                            af = maximum
                    sar[i] = sar[i-1] + (xp - sar[i-1])*af
                    if sar[i] < high[i-1]:
                        sar[i] = high[i-1]
                    if sar[i] < high[i]:
                        sar[i] = high[i]

                        
            else:
                raise ValueError
            


            #sar[i] = SAR_inc(xp, af, sar[i-1])
    print(direction)        
    
    return sar    
            

In [9]:
high

array([27.21613057, 11.05455109, 29.47989371, 67.33815094, 25.51750044,
       53.05875056, 91.86038902, 67.6714435 , 95.86792357, 15.00681479,
       45.3017918 , 43.0969136 , 83.02015322, 45.25862699, 91.83258067,
        1.67618352, 98.29388976, 98.26942135, 43.5923002 , 35.36220755,
       36.46352543,  5.57920418, 82.63839682, 97.23019266, 31.10810285,
       73.41131204, 67.58549767, 10.94676165, 37.4380095 , 77.2540889 ,
       52.17297569, 35.87872867, 36.84370406, 91.61161607, 18.81368632,
       83.64881763, 70.7975417 , 29.12372871,  0.66604967, 15.90576013,
        2.97956941, 76.31341632,  8.15300412, 92.11866932, 32.79505683,
        6.0971182 , 48.95984857, 36.78110879, 51.49392779, 21.31552518,
       55.40714538, 66.49548698, 57.17572671, 79.26543038, 26.27574936,
       77.02120141, 10.16097318, 92.74416668, 89.9958961 , 16.0106075 ,
       49.65950406,  9.18107906, 46.37314192, 76.34477507, 74.20522238,
       96.64315667, 10.26340073, 56.46477758, 93.8307297 , 64.18

In [43]:
low

array([ 4.99178006, 52.24132309, 40.36336318, 30.90618508, 38.02937622,
       54.83330729, 61.48398157,  3.55750497, 83.06198309, 20.02272284,
       16.70049432, 15.85972653, 65.93366005, 79.01748272, 19.6247644 ,
       97.5414912 , 83.29667347, 30.5005054 , 89.65493488, 31.25754062,
       65.49038981, 35.09666783, 45.42740616, 13.63645821, 42.86526495,
       70.37841428, 71.374283  , 93.9363843 ,  4.32835522, 56.49403354,
       21.0150225 , 35.74502259, 91.26673421, 21.98698942,  1.3081256 ,
       86.42059625,  4.6959933 , 66.42448581, 72.92679752, 13.7368291 ,
       99.66236368, 15.99623433, 61.72463917, 96.01570912, 96.41425082,
       56.1589587 , 62.38254541, 87.56317115, 74.82875988, 98.06490349,
       65.88031873, 69.13373126, 11.54129936, 21.15284766,  2.59982408,
       70.87005926, 66.08334224, 12.57130995, 81.5480436 ,  5.33176609,
       92.56558028, 97.50537872, 58.62459054, 24.47766881, 93.26158233,
       90.77917037, 19.7824653 , 16.86368853, 13.77389084, 46.25

In [42]:
SAR(high, low, acceleration=0.02, maximum=0.2) - talib.SAR(high, low, acceleration=0.02, maximum=0.2)

['', 'long', 'long', 'long', 'long', 'long', 'long', 'short', 'long', 'long', 'long', 'long', 'long', 'long', 'long', 'long', 'long', 'long', 'long', 'long', 'long', 'long', 'long', 'short', 'short', 'short', 'short', 'short', 'short', 'short', 'short', 'short', 'short', 'long', 'short', 'short', 'short', 'short', 'short', 'short', 'short', 'short', 'short', 'long', 'long', 'long', 'long', 'long', 'long', 'long', 'long', 'long', 'short', 'short', 'short', 'short', 'short', 'long', 'long', 'short', 'short', 'short', 'short', 'short', 'short', 'long', 'long', 'long', 'long', 'long', 'long', 'long', 'long', 'long', 'short', 'short', 'short', 'short', 'long', 'long', 'long', 'long', 'short', 'short', 'short', 'short', 'short', 'short', 'short', 'short', 'long', 'long', 'long', 'long', 'long', 'long', 'long', 'long', 'long', 'long']


array([        nan,  0.        ,  0.85826913,  3.56560558,  3.35166924,
        3.15056909,  5.9427204 ,  0.        ,  0.        ,  1.84620837,
        1.8092842 ,  1.77309852,  1.73763655,  1.70288382,  1.66882614,
        1.63544962,  3.3025199 ,  3.17041911,  3.04360234,  2.92185825,
        2.80498392,  2.69278456,  1.42960755, -1.0636971 ,  0.        ,
       -1.67187469, -1.6384372 , -1.60566845, -3.51943428, -3.37865691,
       -3.24351064, -3.11377021, -2.9892194 ,  1.74566522,  0.        ,
       -1.80606981, -1.76994841, -1.73454944, -1.69985846, -1.66586129,
       -1.63254406, -1.59989318, -1.56789532,  1.81621087,  1.77988666,
        1.74428892,  1.70940315,  1.67521508,  1.64171078,  1.60887657,
        1.57669903,  1.54516505, -1.6115474 , -1.57931645, -3.45311926,
       -3.31499449, -3.18239471,  1.80288685,  1.76682912, -1.74824801,
       -1.71328305, -1.67901739, -1.64543704, -1.6125283 , -1.58027774,
        1.82622781,  1.78970326,  1.75390919,  1.71883101,  1.35

In [44]:
SAR(high, low, acceleration=0.02, maximum=0.2)

['', 'long', 'long', 'long', 'long', 'long', 'long', 'short', 'long', 'long', 'long', 'long', 'long', 'long', 'long', 'long', 'long', 'long', 'long', 'long', 'long', 'long', 'long', 'short', 'short', 'short', 'short', 'short', 'short', 'short', 'short', 'short', 'short', 'long', 'short', 'short', 'short', 'short', 'short', 'short', 'short', 'short', 'short', 'long', 'long', 'long', 'long', 'long', 'long', 'long', 'long', 'long', 'short', 'short', 'short', 'short', 'short', 'long', 'long', 'short', 'short', 'short', 'short', 'short', 'short', 'long', 'long', 'long', 'long', 'long', 'long', 'long', 'long', 'long', 'short', 'short', 'short', 'short', 'long', 'long', 'long', 'long', 'short', 'short', 'short', 'short', 'short', 'short', 'short', 'short', 'long', 'long', 'long', 'long', 'long', 'long', 'long', 'long', 'long', 'long']


array([        nan,  4.99178006,  5.97130461,  9.65331539, 13.11440552,
       16.36783025, 22.40723495, 91.86038902,  3.55750497,  5.40371335,
        7.21299755,  8.98609607, 10.72373262, 12.42661644, 14.09544258,
       15.7308922 , 19.0334121 , 22.20383121, 25.24743355, 28.1692918 ,
       30.97427572, 33.66706028, 35.09666783, 97.23019266, 97.23019266,
       95.55831797, 93.91988077, 92.31421232, 88.79477804, 85.41612112,
       82.17261049, 79.05884028, 76.06962087,  6.07402043, 91.61161607,
       89.80554626, 88.03559785, 86.3010484 , 84.60118994, 82.93532866,
       81.3027846 , 79.70289142, 78.1349961 ,  3.12433648,  4.90422314,
        6.64851206,  8.35791521, 10.03313029, 11.67484107, 13.28371763,
       14.86041667, 16.40558172, 90.50712192, 88.92780547, 85.47468622,
       82.15969173, 78.97729702,  4.40271093,  6.16954005, 90.99591867,
       89.28263562, 87.60361823, 85.95818119, 84.34565289, 82.76537515,
        7.15799391,  8.94769716, 10.70160635, 12.42043736, 13.77

In [45]:
talib.SAR(high, low, acceleration=0.02, maximum=0.2)

array([        nan,  4.99178006,  5.11303548,  6.08770981,  9.76273628,
       13.21726116, 16.46451455, 91.86038902,  3.55750497,  3.55750497,
        5.40371335,  7.21299755,  8.98609607, 10.72373262, 12.42661644,
       14.09544258, 15.7308922 , 19.0334121 , 22.20383121, 25.24743355,
       28.1692918 , 30.97427572, 33.66706028, 98.29388976, 97.23019266,
       97.23019266, 95.55831797, 93.91988077, 92.31421232, 88.79477804,
       85.41612112, 82.17261049, 79.05884028,  4.32835522, 91.61161607,
       91.61161607, 89.80554626, 88.03559785, 86.3010484 , 84.60118994,
       82.93532866, 81.3027846 , 79.70289142,  1.3081256 ,  3.12433648,
        4.90422314,  6.64851206,  8.35791521, 10.03313029, 11.67484107,
       13.28371763, 14.86041667, 92.11866932, 90.50712192, 88.92780547,
       85.47468622, 82.15969173,  2.59982408,  4.40271093, 92.74416668,
       90.99591867, 89.28263562, 87.60361823, 85.95818119, 84.34565289,
        5.33176609,  7.15799391,  8.94769716, 10.70160635, 12.42

In [57]:
 l = [''] * 20

In [68]:
None * 2

TypeError: unsupported operand type(s) for *: 'NoneType' and 'int'

In [2]:
'true' if False else 'false'

'false'